# Create Future Climate

This script is used to convert future climate projections from the [Ontario Climate Data Portal](https://lamps.math.yorku.ca/OntarioClimate/index_v18.htm) (OCDP) to the Raven climate input data format.

Import packages

In [1]:
import os, shutil
import numpy as np
import scipy
import shapefile
from shapely.geometry import shape
import pandas as pd
from tqdm import tqdm

### Set model output directory, user settings

> Users must change the following input according to the model they wish to convert.

In [2]:
ravenbaselinemodelfile = '../model_baseline/Raven2025.rvi' # model user wishes to apply future climate to
swsfp = "../GIS/CH_Subs_Raven2025.shp"
ocdpfp = "../GIS/OCDP/polygon9864-CH.shp" # do not change
rcp = 'RCP85' # select emission scenario (options: RCP26, RCP45, RCP60, RCP85)
modelname = 'CSIRO-Mk3-6-0' # select model
ravenfuturemodeldir = '../model_future_climate_{}_{}/'.format(modelname,rcp) # directory of future climate Raven directory

Create directories and copy baseline model files

In [3]:
if not os.path.exists(ravenfuturemodeldir):
    os.makedirs(ravenfuturemodeldir)
    os.makedirs(ravenfuturemodeldir+'input')

if not os.path.exists(ravenbaselinemodelfile):
    print('ERROR: model files cannot be found')
else:
    fname = os.path.basename(ravenbaselinemodelfile)[:-4] # get model file name, remove extension
    def copyRaven(ext):
        if not os.path.exists(ravenfuturemodeldir+fname+ext): shutil.copy(ravenbaselinemodelfile[:-4]+ext,ravenfuturemodeldir+fname+ext)
    copyRaven('.bat')
    copyRaven('.rvc')
    copyRaven('.rvh')
    copyRaven('.rvi')
    copyRaven('.rvp')
    copyRaven('.rvt')
    copyRaven('-channels.rvp')
    copyRaven('-lakes.rvh')
    if not os.path.exists(ravenfuturemodeldir+'Raven3.8.exe'): shutil.copy('../scripts/supp/Raven3.8.exe',ravenfuturemodeldir+'Raven3.8.exe')

## Download OCDP data

The first step is to run `scripts/getOCDP.py` to connect with OCDP API and download the future climate projections which come as a set of grid files. *This takes some time (~20 minutes) to complete and downloads roughly 20GB of data per emission scenario.*

In the code that follows, only a subset of grid points that cover the CH model area, data are extracted from matlab grid files (.mat) and converted to csv. 

In [5]:
grdpnts = [8598,8599,8600,8601,8602,8642,8643,8644,8645,8646,8670,8671,8672,8673,8674,8696,8697,8698,8699] # see ../GIS/OCDP/polygon9864-CH.shp

# build mask
msk = np.zeros(8964, dtype=bool)
msk[np.array(grdpnts)-1] = True

# collect OCDP-downloaded matlab grid file (.mat) paths
fps = list()
for root, _, files in os.walk('output/OCDP/'+rcp+'/'):
    for filename in files:
        if filename.lower().endswith('.mat'): fps.append(os.path.join(root, filename))

dtrng = pd.date_range(start="1981-01-01",end="2099-12-31")
dtrng = dtrng[~((dtrng.month == 2) & (dtrng.day == 29))] # ignore leap days (https://github.com/pandas-dev/pandas/issues/56968)
colnams = ["g"+str(i) for i in grdpnts]
gpr, gtn, gtx = None, None, None
for fp in fps:
    fn = os.path.basename(fp)
    stp = fn.split("_")
    par = stp[0]
    mdl = stp[2]
    if mdl != modelname: continue
    print(fn)
    frcp = stp[3]

    mat = scipy.io.loadmat(fp)['outputData'][msk,:,:].astype(np.float32) # load matlab files (https://lamps.math.yorku.ca/OntarioClimate/index_app_documents.htm#/faq)
    mat/=10. # data are converted from integers to decimals
    mat = mat.transpose(0,2,1).reshape(len(grdpnts), -1)

    df = pd.DataFrame(mat.T, columns=colnams, index=dtrng)
    df.index.name='Date'
    # df.to_csv(ravenfuturemodeldir+'dat/'+fn+".csv")
    
    if par=='pr':
        gpr = df
    elif par=='tasmin':
        gtn = df
    elif par=='tasmax':
        gtx = df

pr_LAMPS_CSIRO-Mk3-6-0_RCP85_8964X365X119.mat
tasmax_LAMPS_CSIRO-Mk3-6-0_RCP85_8964X365X119.mat
tasmin_LAMPS_CSIRO-Mk3-6-0_RCP85_8964X365X119.mat


## Perform overlay analysis

this block will overlay the CH sub-watersheds and the OCDP grid in order to determine the proportion with which the OCDP layer covers each sub-watersheds. This proportion will then be used to weight future climate imports accordingly.

In [6]:
def getGeoms(fp,anam):
    sf = shapefile.Reader(fp)
    shp = sf.shapes()
    attr = sf.records()
    return [shape(s.__geo_interface__) for s in shp], [a[anam] for a in attr]

sws_geom, sws_id = getGeoms(swsfp,'SubId') # import Raven subwatersheds
ocdp_geom, ocdp_id =  getGeoms(ocdpfp,'id') # import OCDP overlay grid

coll = []
for i, gsws in enumerate(sws_geom):
    asws = gsws.area
    for j, gocdp in enumerate(ocdp_geom):
        intersection = gsws.intersection(gocdp)
        if not intersection.is_empty:
            intersection_area = intersection.area
            proportion = intersection_area / asws if asws > 0 else 0
            coll.append({
                'SubId': sws_id[i],
                'OCDPid': 'g'+str(ocdp_id[j]),
                'intersectArea': intersection_area,
                'swsArea': asws,
                'proportion': proportion
            })

dfw = pd.DataFrame(coll)
# dfw.to_csv('Raven_to_OCDP_intersection_proportions.csv', index=False)

Create function to write Raven input files

In [7]:
def writeRvt(fp,df):
    with open(fp,"w") as f:
        f.write(':MultiData\n')
        f.write(' {} {} {}\n'.format(df.index[0].strftime("%Y-%m-%d %H:%M:%S"), 1, len(df.index)))        
        f.write(' :Parameters  TEMP_MAX  TEMP_MIN    PRECIP\n')
        f.write(' :Units              C         C      mm/d\n')
        for _, row in df.iterrows():
            f.write('            {:>10.2f}{:>10.2f}{:>10.1f}\n'.format(round(row['tx'],2)+0, round(row['tn'],2)+0, row['pr']))
        f.write(':EndMultiData\n\n')

Collect weighted data, write to Raven input files

In [8]:
def weightData(sid):
    sdfw = dfw[dfw['SubId']==sid]
    wghts = dict(zip(sdfw.OCDPid,sdfw.proportion))
    wpr = (gpr[wghts.keys()].mul(wghts)).sum(1)
    wtn = (gtn[wghts.keys()].mul(wghts)).sum(1)
    wtx = (gtx[wghts.keys()].mul(wghts)).sum(1)
    writeRvt(ravenfuturemodeldir+'input/m{}.rvt'.format(sid), pd.DataFrame([wpr,wtn,wtx]).T.rename(columns={0:'pr',1:'tn',2:'tx'}))

for sid in set(dfw['SubId']): weightData(sid)

Lastly, a couple changes need to be made to the Raven input files

First, adjust date ranges in the .rvi file, for example:

```r
:StartDate 1981-10-01 00:00:00
:EndDate   2099-09-30 00:00:00
```

> Note: ODCP daily data ranges from 1981-01-01 to 2099-12-31

<br>

Next, __*remove*__ the following lines from the .rvt file:

```python
# Observing outlet at subbasin 48 to file: 02HB022.csv 
:RedirectToFile input\g02hb022.rvt

# Observing outlet at subbasin 54 to file: 02HB028.csv 
:RedirectToFile input\g02hb028.rvt

# Observing outlet at subbasin 55 to file: 02HB012.csv 
:RedirectToFile input\g02hb012.rvt

# Observing outlet at subbasin 56 to file: 02HB005.csv 
:RedirectToFile input\g02hb005.rvt

# Observing outlet at subbasin 57 to file: 02HB004.csv 
:RedirectToFile input\g02hb004.rvt

# Observing outlet at subbasin 58 to file: 02HB027.csv 
:RedirectToFile input\g02hb027.rvt

# Observing outlet at subbasin 60 to file: 02HB011.csv 
:RedirectToFile input\g02hb011.rvt

# Observing outlet at subbasin 63 to file: 02HB033.csv 
:RedirectToFile input\g02hb033.rvt

# Observing outlet at subbasin 64 to file: 02HB032.csv 
:RedirectToFile input\g02hb032.rvt
```